In [1]:
#hide
#default_exp lookup
#default_cls_lvl 3

In [2]:
#export
from nbdev.imports import *
from nbdev.read import *
from nbdev.export import *
from nbdev.doclinks import *
from fastcore.utils import *

import pkg_resources,importlib

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page
else: Markdown,display,page = None,None,None

In [3]:
from fastcore.test import *

# Doc lookup

> Convert backticks to links

In [4]:
#export
class NbdevLookup:
    "Mapping from symbol names to URLs with docs"
    def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None):
        skip_mods = setify(skip_mods)
        strip_libs = L(strip_libs)
        if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
        # Dict from lib name to _nbdev module for incl_libs (defaults to all)
        self.entries = {o.dist.key:o.load() for o in pkg_resources.iter_entry_points(group='nbdev')
                       if incl_libs is None or o.dist.key in incl_libs}
        py_syms = merge(*L(o.modidx['syms'].values() for o in self.entries.values()).concat())
        for m in strip_libs:
            _d = self.entries[m].modidx
            stripped = {remove_prefix(k,f"{mod}."):v
                        for mod,dets in _d['syms'].items() if mod not in skip_mods
                        for k,v in dets.items()}
            py_syms = merge(stripped, py_syms)
        self.syms = py_syms

    def __getitem__(self, s): return self.syms.get(s, None)

Symbol names are taken from libraries registered using the 'nbdev' entry point. By default, all libraries with this entry point are searched, but full symbol names (including module prefix) are required.

In [5]:
c = NbdevLookup()
assert c['nbdev.doclinks.DocLinks'].startswith('http')
assert c['numpy.array'].startswith('http')
assert not c['DocLinks']

Pass `strip_libs` to list libraries which should be available without requiring a module prefix.

In [6]:
c = NbdevLookup(strip_libs=['nbdev','nbdev-numpy'])
assert c['DocLinks'].startswith('http')
assert c['array'].startswith('http')

In [7]:
#export
from nbdev._nbdev import modidx
_settings = modidx['settings']
_strip_libs  = _settings.get('strip_libs',_settings.get('lib_name')).split()
nbdev_lookup = NbdevLookup(_strip_libs)

nbdev itself includes `nbdev_lookup`, an instantiated `NbdevLookup` with `strip_libs=nbdev`.

In [8]:
assert nbdev_lookup['DocLinks'].startswith('http')
assert nbdev_lookup['numpy.array'].startswith('http')
assert not nbdev_lookup['array']

## Backticks

In [9]:
#export
@patch
def _link_sym(self:NbdevLookup, m):
    l = m.group(1)
    s = self[l]
    if s is None: return m.group(0)
    return rf"[{l}]({s})"

_re_backticks = re.compile(r'`([^`\s]+)`')
@patch
def _link_line(self:NbdevLookup, l): return _re_backticks.sub(self._link_sym, l)

@patch
def linkify(self:NbdevLookup, md):
    in_fence=False
    lines = md.splitlines()
    for i,l in enumerate(lines):
        if l.startswith("```"): in_fence=not in_fence
        elif not l.startswith('    ') and not in_fence: lines[i] = self._link_line(l)
    return '\n'.join(lines)

In [10]:
md = """This is a link to `numpy.array` and to `read_nb` but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `read_nb`

```
This isn't a link to `read_nb` either
```"""

In [11]:
c = NbdevLookup('nbdev')
Markdown(c.linkify(md))

This is a link to [numpy.array](https://numpy.org/doc/stable/reference/generated/numpy.array.html#numpy.array) and to [read_nb](https://nbdev.fast.ai/nbdev.read#read_nb) but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `read_nb`

```
This isn't a link to `read_nb` either
```

## Export -

In [12]:
from nbdev.doclinks import nbdev_build_lib
nbdev_build_lib()